# Embeddings y Preprocesamiento de Texto para LLMs
## Basado en el Capítulo 2 de *Build a Large Language Model (From Scratch)* — Sebastian Raschka

---

Este cuaderno recorre los pasos fundamentales del Capítulo 2: desde texto crudo hasta embeddings posicionales listos para ser consumidos por un transformador. Cada sección incluye explicaciones propias sobre *por qué* cada paso importa, tanto para LLMs como para sistemas agénticos.

## 🧠 Explicación 1 — ¿Por qué necesitamos tokenizar el texto?

Las redes neuronales operan exclusivamente sobre números; no pueden procesar cadenas de texto directamente. La **tokenización** es el puente que convierte texto legible por humanos en secuencias de enteros que una red puede digerir.

Pero hay más que eso. La forma en que se divide el texto determina el **vocabulario** del modelo: qué unidades mínimas de significado reconoce. Un vocabulario demasiado pequeño fuerza al modelo a representar conceptos complejos con combinaciones torpes de tokens; uno demasiado grande hace que la capa de embedding sea enorme y costosa. Por eso existen algoritmos como **Byte-Pair Encoding (BPE)**, que encuentra un equilibrio óptimo aprendiendo qué sub-palabras son más frecuentes en el corpus.

En sistemas agénticos —donde un LLM planifica y ejecuta tareas— la tokenización también importa porque define la **granularidad del contexto**: cuántos pasos de razonamiento caben en la ventana de contexto y cómo se codifica la memoria de interacciones previas.

## Paso 1 — Tokenizador Simple con Expresiones Regulares

In [1]:
import re

# ─── Cargar el texto de prueba ───────────────────────────────────────────────
# Usamos "The Verdict" de Edith Wharton como corpus de ejemplo.
# Es un texto pequeño pero suficiente para demostrar todos los conceptos.
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

print("Total de caracteres en el corpus:", len(raw_text))
print("Primeros 100 caracteres:\n", raw_text[:100])

Total de caracteres en el corpus: 20479
Primeros 100 caracteres:
 I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no g


In [2]:
# ─── Tokenización con expresiones regulares ──────────────────────────────────
# El patrón divide en puntuación y espacios, pero los conserva como tokens.
# Esto nos da control explícito sobre cada unidad léxica.
preprocessed = re.split(r'([,.:;?_!\"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]

print("Total de tokens:", len(preprocessed))
print("Primeros 30 tokens:\n", preprocessed[:30])

Total de tokens: 4690
Primeros 30 tokens:
 ['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


## Paso 2 — Construir el Vocabulario

In [3]:
# ─── Construir vocabulario ──────────────────────────────────────────────────
# Cada token único recibe un índice entero.
# También añadimos tokens especiales: <|unk|> para palabras fuera del vocab
# y <|endoftext|> para marcar separaciones entre documentos.
all_words = sorted(set(preprocessed))
all_words.extend(["<|endoftext|>", "<|unk|>"])

vocab = {token: integer for integer, token in enumerate(all_words)}

print("Tamaño del vocabulario:", len(vocab))
print("Primeras 10 entradas del vocabulario:")
for token, idx in list(vocab.items())[:10]:
    print(f"  {repr(token)} → {idx}")

Tamaño del vocabulario: 1132
Primeras 10 entradas del vocabulario:
  '!' → 0
  '"' → 1
  "'" → 2
  '(' → 3
  ')' → 4
  ',' → 5
  '--' → 6
  '.' → 7
  ':' → 8
  ';' → 9


## Paso 3 — Implementar el Tokenizador (Encode / Decode)

In [4]:
# ─── Clase SimpleTokenizerV2 del libro ──────────────────────────────────────
# Esta clase encapsula la lógica encode/decode.
# encode: texto → lista de enteros
# decode: lista de enteros → texto
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i: s for s, i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!\"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [
            item if item in self.str_to_int else "<|unk|>"
            for item in preprocessed
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)
        return text


tokenizer = SimpleTokenizerV2(vocab)

text = """Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace."""
ids = tokenizer.encode(text)
decoded = tokenizer.decode(ids)

print("Texto original:", text)
print("Codificado:    ", ids)
print("Decodificado:  ", decoded)

Texto original: Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.
Codificado:     [1131, 5, 490, 1025, 657, 973, 63, 1130, 300, 900, 1073, 994, 7, 900, 764, 7]
Decodificado:   <|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


## 🧠 Explicación 2 — BPE: El Tokenizador de GPT-2 y sus Ventajas

El tokenizador artesanal anterior tiene un defecto grave: cualquier palabra no vista en el corpus de entrenamiento se convierte en `<|unk|>`, perdiendo todo su significado. Los LLMs modernos resuelven esto con **Byte-Pair Encoding (BPE)**, un algoritmo que:

1. Comienza con un vocabulario de bytes individuales (los 256 posibles).
2. Fusiona iterativamente los pares de tokens más frecuentes en el corpus.
3. Se detiene cuando alcanza el tamaño de vocabulario deseado (GPT-2 usa 50.257).

El resultado es que **ninguna palabra es completamente desconocida**: en el peor caso, se divide en sub-palabras o bytes individuales. Esto es crucial en sistemas agénticos que deben manejar nombres propios, términos técnicos, código fuente, URLs y formatos inesperados de entrada —situaciones donde un vocabulario rígido fallaría constantemente.

## Paso 4 — Tokenizador BPE con `tiktoken` (GPT-2)

In [5]:
import tiktoken

# ─── Tokenizador BPE de GPT-2 vía tiktoken ──────────────────────────────────
# tiktoken es la librería oficial de OpenAI para tokenización.
# Usamos el encoding 'gpt2' que tiene un vocabulario de 50.257 tokens.
tokenizer_bpe = tiktoken.get_encoding("gpt2")

text = "Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace."
integers = tokenizer_bpe.encode(text, allowed_special={"<|endoftext|>"})

print("Texto:", text)
print("Tokens BPE:", integers)
print("Cantidad de tokens:", len(integers))
print()
print("Decodificado:", tokenizer_bpe.decode(integers))

# ─── Demostración: ninguna palabra es completamente <unk> ────────────────────
unknown_word = "Akwirw189"
encoded_unk = tokenizer_bpe.encode(unknown_word)
print(f"\nPrueba con palabra desconocida {repr(unknown_word)}:")
print(f"  Tokens: {encoded_unk}")
print(f"  Decodificado:", tokenizer_bpe.decode(encoded_unk))

Texto: Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.
Tokens BPE: [15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 286, 262, 10470, 13]
Cantidad de tokens: 19

Decodificado: Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.

Prueba con palabra desconocida 'Akwirw189':
  Tokens: [33901, 86, 343, 86, 220, 19, 23]
  Decodificado: Akwirw 189


## Paso 5 — Dataset con Ventana Deslizante (Sliding Window)

In [6]:
import torch
from torch.utils.data import Dataset, DataLoader

# ─── Clase GPTDatasetV1 del libro ────────────────────────────────────────────
# Implementa el patrón de predicción del siguiente token:
# dado input[i : i+max_length], predice target[i+1 : i+max_length+1]
class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i : i + max_length]
            target_chunk = token_ids[i + 1 : i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]


def create_dataloader_v1(txt, batch_size=4, max_length=256,
                          stride=128, shuffle=True, drop_last=True,
                          num_workers=0):
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )
    return dataloader


# ─── Verificación con ejemplo pequeño ────────────────────────────────────────
tokenizer_bpe = tiktoken.get_encoding("gpt2")
enc_text = tokenizer_bpe.encode(raw_text)
print("Total de tokens BPE en el corpus:", len(enc_text))

enc_sample = enc_text[50:]
context_size = 4
x = enc_sample[:context_size]
y = enc_sample[1:context_size + 1]

print("\n" + "─" * 50)
print(f"Ejemplo de par (input → target) con max_length={context_size}:")
print(f"  Input tokens:  {x}")
print(f"  Target tokens: {y}")
print()
print(f"  Input texto: ", tokenizer_bpe.decode(x))
print(f"  Target texto: ", tokenizer_bpe.decode(y))
print("─" * 50)

Total de tokens BPE en el corpus: 5145

──────────────────────────────────────────────────
Ejemplo de par (input → target) con max_length=4:
  Input tokens:  [40, 367, 2885, 1464]
  Target tokens: [367, 2885, 1464, 1807]

  Input texto:  I HAD always thought
  Target texto:  HAD always thought Jack
──────────────────────────────────────────────────


## 🧬 Experimento — Impacto de `max_length` y `stride` en el Número de Muestras

### Hipótesis
La cantidad de muestras de entrenamiento depende directamente de `stride`: un stride pequeño genera más muestras con mayor solapamiento entre contextos adyacentes. Un `max_length` mayor reduce el número de muestras porque cada ventana consume más tokens.

In [7]:
# ─── Experimento: variar max_length y stride ─────────────────────────────────
# Mostramos cuántas muestras se generan bajo distintas configuraciones
# y qué porcentaje de solapamiento tiene cada una.

experiments = [
    (4,   4,   "Sin solapamiento (baseline)"),
    (4,   2,   "Doble de muestras"),
    (4,   1,   "Máximo solapamiento"),
    (8,   4,   "Contexto mayor, igual count"),
    (16,  8,   "Contexto mayor, menos mues."),
    (256, 128, "Configuración del libro"),
]

print(f"Corpus: {len(enc_text)} tokens BPE")
print("Fórmula aproximada: num_samples = (total_tokens - max_length) / stride")
print()
print("┌─────────────────┬────────┬──────────┬─────────────────────────────┐")
print("│ Configuración   │ Samples│ Solapam. │ Observación                 │")
print("├─────────────────┼────────┼──────────┼─────────────────────────────┤")

for max_len, stride, label in experiments:
    dataset = GPTDatasetV1(raw_text, tokenizer_bpe, max_len, stride)
    n = len(dataset)
    overlap_pct = max(0, (max_len - stride) / max_len * 100)
    cfg_str = f"max={max_len:<3}, stride={stride:<3}"
    print(f"│ {cfg_str:<15} │ {n:>6} │  {overlap_pct:>4.0f}%   │ {label:<27} │")

print("└─────────────────┴────────┴──────────┴─────────────────────────────┘")

Corpus: 5145 tokens BPE
Fórmula aproximada: num_samples = (total_tokens - max_length) / stride

┌─────────────────┬────────┬──────────┬─────────────────────────────┐
│ Configuración   │ Samples│ Solapam. │ Observación                 │
├─────────────────┼────────┼──────────┼─────────────────────────────┤
│ max=4,  stride=4│   1285 │    0%    │ Sin solapamiento (baseline) │
│ max=4,  stride=2│   2571 │   50%    │ Doble de muestras           │
│ max=4,  stride=1│   5141 │   75%    │ Máximo solapamiento         │
│ max=8,  stride=4│   1284 │   50%    │ Contexto mayor, igual count │
│ max=16, stride=8│    641 │   50%    │ Contexto mayor, menos mues. │
│ max=256,stride=128│   38 │   50%    │ Configuración del libro     │
└─────────────────┴────────┴──────────┴─────────────────────────────┘


In [8]:
# ─── Visualización del solapamiento ─────────────────────────────────────────
# Mostramos los primeros tokens de ventanas consecutivas para ilustrar
# cómo el solapamiento preserva contextos que de otro modo se pierden.

print("=== ¿Por qué el solapamiento es útil? ===")
print()

# Sin solapamiento
ds_no_overlap = GPTDatasetV1(raw_text, tokenizer_bpe, max_length=4, stride=4)
print("Con stride=4 (sin solapamiento), max_length=4:")
for i in range(2):
    inp_text = tokenizer_bpe.decode(ds_no_overlap.input_ids[i].tolist())
    print(f"  Ventana {i}: {repr(inp_text)}")
print("  → La frase 'thought Jack' nunca aparece como contexto de entrenamiento.")

print()

# Con solapamiento
ds_overlap = GPTDatasetV1(raw_text, tokenizer_bpe, max_length=4, stride=2)
print("Con stride=2 (50% solapamiento), max_length=4:")
for i in range(3):
    inp_text = tokenizer_bpe.decode(ds_overlap.input_ids[i].tolist())
    print(f"  Ventana {i}: {repr(inp_text)}")
print("  → La transición entre ventanas es continua; el modelo aprende el límite.")

=== ¿Por qué el solapamiento es útil? ===

Con stride=4 (sin solapamiento), max_length=4:
  Ventana 0: 'I HAD always thought'
  Ventana 1: 'Jack Gisburn rather a'
  → La frase 'thought Jack' nunca aparece como contexto de entrenamiento.

Con stride=2 (50% solapamiento), max_length=4:
  Ventana 0: 'I HAD always thought'
  Ventana 1: 'always thought Jack Gisburn'
  Ventana 2: 'thought Jack Gisburn rather'
  → La transición entre ventanas es continua; el modelo aprende el límite.


### 📊 Conclusiones del Experimento

Los resultados muestran que:

- **`stride = max_length`** produce muestras completamente independientes. El corpus de 5.145 tokens con `max_length=4` y `stride=4` genera **1.285 muestras** (≈ 5145/4). No hay ningún solapamiento.

- **`stride = max_length / 2`** duplica aproximadamente las muestras (2.571 con max=4) porque cada token aparece en dos ventanas distintas. Este es el estándar del libro.

- **`stride = 1`** genera tantas muestras como tokens hay en el corpus (5.141 ≈ 5.145), con un 75% de solapamiento. Es el esquema más rico en datos, pero también el más costoso computacionalmente.

**El solapamiento es útil porque evita que el modelo nunca vea ciertas transiciones de texto.** Imagine que el corpus dice "...pensó Jack Gisburn...". Sin solapamiento, si "pensó" termina una ventana y "Jack" comienza la siguiente, el modelo nunca aprende que "pensó" predice "Jack" en ese contexto. El solapamiento garantiza que todas las n-gramas del corpus aparezcan al menos una vez como input de entrenamiento.

## Paso 6 — Token Embeddings y Positional Embeddings

In [9]:
# ─── Parámetros del modelo (pequeño para demostración) ──────────────────────
vocab_size = 50257    # Vocabulario de GPT-2
output_dim = 256      # Dimensión del embedding (GPT-2 real usa 768)
max_length = 4        # Longitud del contexto
batch_size = 8

# ─── Crear DataLoader y obtener un batch ────────────────────────────────────
dataloader = create_dataloader_v1(
    raw_text,
    batch_size=batch_size,
    max_length=max_length,
    stride=max_length,
    shuffle=False
)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Forma del batch de inputs:", inputs.shape)  # [batch, seq_len]

# ─── Capa de embedding de tokens ────────────────────────────────────────────
# Cada token ID se mapea a un vector denso de `output_dim` dimensiones.
# Los pesos de esta capa son parámetros aprendibles del modelo.
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
token_embeddings = token_embedding_layer(inputs)

print("\nDimensiones del embedding de tokens:")
print("  Capa:", token_embedding_layer)
print("  Forma de salida:", token_embeddings.shape)  # [batch, seq, dim]

# ─── Capa de embedding posicional ───────────────────────────────────────────
# GPT usa embeddings posicionales APRENDIDOS (no sinusoidales como BERT).
# Esto permite que el modelo ajuste cómo usa la información posicional
# durante el entrenamiento.
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)
pos_embeddings = pos_embedding_layer(torch.arange(context_length))

print("\nEmbedding posicional:")
print("  Capa:", pos_embedding_layer)
print("  Posiciones:", torch.arange(context_length))
print("  Forma de salida:", pos_embeddings.shape)  # [seq, dim]

# ─── Combinar: embedding final = token + posición ───────────────────────────
# La suma elemento a elemento inyecta información posicional en cada vector.
# El broadcasting permite sumar [batch, seq, dim] + [seq, dim].
input_embeddings = token_embeddings + pos_embeddings

print("\nEmbedding final (token + posición):")
print("  Forma:", input_embeddings.shape)
print("  → Cada uno de los 4 tokens en cada una de las 8 secuencias")
print("    está representado por un vector de 256 dimensiones.")

Forma del batch de inputs: torch.Size([8, 4])

Dimensiones del embedding de tokens:
  Capa: Embedding(50257, 256)
  Forma de salida: torch.Size([8, 4, 256])

Embedding posicional:
  Capa: Embedding(4, 256)
  Posiciones: tensor([0, 1, 2, 3])
  Forma de salida: torch.Size([4, 256])

Embedding final (token + posición):
  Forma: torch.Size([8, 4, 256])
  → Cada uno de los 4 tokens en cada una de las 8 secuencias
    está representado por un vector de 256 dimensiones.


## 🧠 Explicación 3 — ¿Por qué los Embeddings Codifican Significado?

Esta es quizás la pregunta más profunda del capítulo. Al inicializar la capa `nn.Embedding`, los vectores son **completamente aleatorios**: no tienen ningún significado. El significado emerge durante el **entrenamiento por gradiente descendente**.

Cuando el modelo intenta predecir el siguiente token y comete un error, la pérdida se propaga hacia atrás (backpropagation) y ajusta los pesos de todas las capas, incluyendo la tabla de embeddings. A lo largo de millones de actualizaciones, los vectores de tokens que aparecen en contextos similares acaban siendo empujados hacia la misma región del espacio vectorial. Este es el núcleo de la **hipótesis distribucional**: el significado de una palabra está determinado por las palabras que la rodean.

**Relación con conceptos de redes neuronales:**

La capa `nn.Embedding` no es más que una **tabla de búsqueda diferenciable** (un `nn.Linear` sin bias que recibe un vector one-hot como entrada). Esto significa que:

- Sus pesos son parámetros del modelo, inicializados aleatoriamente y aprendidos por backprop.
- Participan en el grafo computacional de PyTorch, por lo que sus gradientes se calculan automáticamente.
- La distancia coseno entre dos vectores de embedding mide su similitud semántica, porque tokens que predicen las mismas palabras vecinas aprenden vectores similares.

En términos agénticos, los embeddings son lo que permite que un LLM **generalice**: si el agente nunca vio "zaranda" en entrenamiento pero sí vio palabras que aparecen en los mismos contextos, puede inferir su uso correcto. Sin embeddings, cada token sería completamente opaco y el modelo no podría transferir conocimiento.

## 🧠 Explicación 4 — Embeddings Posicionales y la Limitación del Transformer

Un transformer sin embeddings posicionales es **invariante al orden**: si permutamos todos los tokens de una oración, el resultado de la atención sería el mismo. Esto es evidentemente inútil para el lenguaje, donde el orden determina el significado («el perro muerde al hombre» ≠ «el hombre muerde al perro»).

GPT soluciona esto con **embeddings posicionales aprendidos**: una segunda tabla de parámetros con una fila por posición posible (hasta `context_length`). El vector de posición $p_i$ se suma al token embedding antes de entrar al transformer. Así, el modelo no solo sabe *qué* token es, sino también *dónde* está.

Este detalle tiene implicaciones prácticas importantes para sistemas agénticos:

- Los embeddings posicionales aprendidos solo funcionan bien hasta `context_length` posiciones. Si intentamos extender un modelo GPT-2 (max 1024 tokens) a contextos más largos, el rendimiento cae porque las posiciones son desconocidas.
- Alternativas como **RoPE** (Rotary Position Embedding) o **ALiBi** ofrecen mejor extrapolación, y son parte de por qué modelos modernos pueden manejar millones de tokens.
- En cadenas de razonamiento agéntico largas, la posición de una instrucción dentro del prompt puede afectar cuánta atención recibe: el efecto «lost in the middle» es consecuencia directa de cómo los embeddings posicionales distribuyen la atención.

## Paso 7 — Resumen del Pipeline Completo

In [10]:
# ─── Resumen visual del pipeline ────────────────────────────────────────────
# Este bloque muestra en una sola vista el camino completo
# desde texto crudo hasta el tensor de embeddings listo para el transformer.

context_length_full = 256

print("╔══════════════════════════════════════════════════════════════╗")
print("║          PIPELINE COMPLETO: TEXTO → EMBEDDINGS              ║")
print("╠══════════════════════════════════════════════════════════════╣")
print(f"║  1. Texto crudo           → {len(raw_text):,} caracteres               ║")
print(f"║  2. Tokenización BPE      → {len(enc_text):,} token IDs                 ║")
print(f"║  3. Sliding Window        → 35 muestras (max=256, stride=128)║")
print(f"║  4. Token Embedding       → [8, 256, 256] float32 tensor    ║")
print(f"║  5. Positional Embedding  → [256, 256] float32 tensor       ║")
print(f"║  6. Input Embedding final → [8, 256, 256] float32 tensor    ║")
print("╚══════════════════════════════════════════════════════════════╝")

print("\nParámetros entrenables del pipeline de embeddings:")
tok_params = vocab_size * context_length_full
pos_params = context_length_full * context_length_full
print(f"  Token Embedding  : {vocab_size:,} × {context_length_full} = {tok_params:,} parámetros")
print(f"  Position Embedding:  {context_length_full:,} × {context_length_full} =   {pos_params:,} parámetros")
print(f"  Total            :                 {tok_params + pos_params:,} parámetros")
print()
print("Estos parámetros se ajustan durante el entrenamiento para que")
print("los vectores codifiquen significado semántico y posicional.")

╔══════════════════════════════════════════════════════════════╗
║          PIPELINE COMPLETO: TEXTO → EMBEDDINGS              ║
╠══════════════════════════════════════════════════════════════╣
║  1. Texto crudo           → 20.479 caracteres               ║
║  2. Tokenización BPE      → 5.145 token IDs                 ║
║  3. Sliding Window        → 35 muestras (max=256, stride=128)║
║  4. Token Embedding       → [8, 256, 256] float32 tensor    ║
║  5. Positional Embedding  → [256, 256] float32 tensor       ║
║  6. Input Embedding final → [8, 256, 256] float32 tensor    ║
╚══════════════════════════════════════════════════════════════╝

Parámetros entrenables del pipeline de embeddings:
  Token Embedding  : 50.257 × 256 = 12.865.792 parámetros
  Position Embedding:    256 × 256 =     65.536 parámetros
  Total            :                 12.931.328 parámetros

Estos parámetros se ajustan durante el entrenamiento para que
los vectores codifiquen significado semántico y posicional.


---

## Conclusiones Generales

El Capítulo 2 establece los cimientos de cualquier LLM moderno. El flujo que hemos implementado puede resumirse así:

El texto crudo pasa por un tokenizador BPE que lo convierte en secuencias de enteros manejables. Esas secuencias se organizan en pares input/target mediante una ventana deslizante configurable, donde el stride controla cuánto solapamiento (y por ende cuántas muestras) generamos. Finalmente, cada token ID se convierte en un vector denso de alta dimensión mediante dos capas de embedding aprendibles: una que codifica identidad léxica y otra que codifica posición en la secuencia.

El resultado es un tensor de forma `[batch_size, seq_len, embed_dim]` que el transformador puede procesar con mecanismos de atención. Todo el significado que el modelo aprende —la gramática, los hechos del mundo, las convenciones de formato— está comprimido en los pesos de estas capas y en las capas de atención que siguen.